# Hohmann Transfer Orbit Visualization

This tutorial demonstrates how to generate satellite Keplerian orbits to visualize a Hohmann transfer orbit around Earth

## Setup

In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go

from ostk.physics.units import Length
from ostk.physics.units import Angle
from ostk.physics.time import Scale
from ostk.physics.time import Instant
from ostk.physics.time import Duration
from ostk.physics.time import Interval
from ostk.physics.time import DateTime
from ostk.physics.coordinate.spherical import LLA
from ostk.physics.coordinate import Frame
from ostk.physics import Environment
from ostk.physics.environment.objects.celestial_bodies import Earth

from ostk.astrodynamics import Trajectory
from ostk.astrodynamics.trajectory import Orbit
from ostk.astrodynamics.trajectory.orbit.models import Kepler
from ostk.astrodynamics.trajectory.orbit.models.kepler import COE
from ostk.astrodynamics.trajectory.orbit.models import SGP4
from ostk.astrodynamics.trajectory.orbit.models.sgp4 import TLE

---

In [2]:
def propagate_kepler_orbit (coe, epoch, earth, interval):
    
    keplerian_model = Kepler(coe, epoch, earth, Kepler.PerturbationType.No)
    orbit = Orbit(keplerian_model, earth)
    
    states = [[instant, orbit.get_state_at(instant)] for instant in instants]
    orbit_data = [convert_state(instant, state) for [instant, state] in states]
    
    orbit_df = pd.DataFrame(
        orbit_data, 
        columns = [
            '$Time^{UTC}$', 
            '$MJD^{UTC}$', 
            '$x_{x}^{ECI}$', 
            '$x_{y}^{ECI}$', 
            '$x_{z}^{ECI}$', 
            '$v_{x}^{ECI}$', 
            '$v_{y}^{ECI}$', 
            '$v_{z}^{ECI}$', 
            '$Latitude$', 
            '$Longitude$', 
            '$Altitude$'
        ]
    )
    
    return orbit_df

In [3]:
def convert_state (instant, state):
    
    lla = LLA.cartesian(
        state.get_position().in_frame(Frame.ITRF(), state.get_instant()).get_coordinates(), 
        Earth.equatorial_radius, 
        Earth.flattening
    )
    
    return [
                repr(instant),
                float(instant.get_modified_julian_date(Scale.UTC)),
                *state.get_position().get_coordinates().tolist(),
                *state.get_velocity().get_coordinates().tolist(),
                float(lla.get_latitude().in_degrees()),
                float(lla.get_longitude().in_degrees()),
                float(lla.get_altitude().in_meters())
            ]

## Keplerian Orbits

In [ ]:
environment = Environment.default()

In [ ]:
epoch = Instant.date_time(DateTime(2022, 1, 1, 0, 0, 0), Scale.UTC)
earth = environment.access_celestial_object_with_name('Earth')

In [ ]:
start_instant = Instant.date_time(DateTime(2022, 1, 1, 0, 0, 0), Scale.UTC)
end_instant = Instant.date_time(DateTime(2022, 1, 2, 0, 0, 0), Scale.UTC)

In [ ]:
interval = Interval.closed(start_instant, end_instant)

In [ ]:
step = Duration.minutes(1.0)

In [ ]:
instants = interval.generate_grid(step)

### Initial Orbit

In [ ]:
a_initial = Length.kilometers(6700.0)
e_initial = 0.0001
i_initial = Angle.degrees(35.0)
raan_initial = Angle.degrees(40.0)
aop_initial = Angle.degrees(45.0)
nu_initial = Angle.degrees(50.0)

initial_coe = COE(a_initial, e_initial, i_initial, raan_initial, aop_initial, nu_initial)

In [ ]:
initial_orbit_df = propagate_kepler_orbit(initial_coe, epoch, earth, interval)

### Final Orbit

In [ ]:
a_final = Length.kilometers(42238.0)
e_final = 0.0001
i_final = Angle.degrees(35.0)
raan_final = Angle.degrees(40.0)
aop_final = Angle.degrees(45.0)
nu_final = Angle.degrees(50.0)

final_coe = COE(a_final, e_final, i_final, raan_final, aop_final, nu_final)

In [ ]:
final_orbit_df = propagate_kepler_orbit(final_coe, epoch, earth, interval)

### Transfer Orbit

In [ ]:
a = Length.kilometers((a_final.in_kilometers() + a_initial.in_kilometers()) / 2)
e = 0.7265
i = Angle.degrees(35.0)
raan = Angle.degrees(40.0)
aop = Angle.degrees(45.0)
nu = Angle.degrees(50.0)

transfer_coe = COE(a, e, i, raan, aop, nu)

In [ ]:
transfer_orbit_df = propagate_kepler_orbit(transfer_coe, epoch, earth, interval)

### Output

3D plot, in **Earth Fixed** frame:

In [ ]:
figure = go.Figure(
    data = [
        go.Scattergeo(
            lon = initial_orbit_df['$Longitude$'],
            lat = initial_orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
                color = 'rgb(255, 62, 79)'
            )
        ),
        go.Scattergeo(
            lon = final_orbit_df['$Longitude$'],
            lat = final_orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
                color = 'rgb(62, 255, 79)'
            )
        ),
        go.Scattergeo(
            lon = transfer_orbit_df['$Longitude$'],
            lat = transfer_orbit_df['$Latitude$'],
            mode = 'lines',
            line = go.scattergeo.Line(
                width = 2,
                color = 'rgb(79, 62, 255)'
            )
        )
    ],
    layout = go.Layout(
        title = None,
        showlegend = False,
        width = 800,
        height = 800,
        geo = go.layout.Geo(
            showland = True,
            showlakes = True,
            showcountries = False,
            showocean = True,
            countrywidth = 0.0,
            landcolor = 'rgb(100, 100, 100)',
            lakecolor = 'rgb(240, 240, 240)',
            oceancolor = 'rgb(240, 240, 240)',
            projection = dict( 
                type = 'orthographic',
                rotation = dict(
                    lon = -100,
                    lat = 40,
                    roll = 0
                )            
            ),
            lonaxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            ),
            lataxis = dict( 
                showgrid = True,
                gridcolor = 'rgb(102, 102, 102)',
                gridwidth = 0.5
            )
        )
    )
)

figure.show()

3D plot, in **Earth Inertial** frame:

In [ ]:
theta = np.linspace(0, 2 * np.pi, 30)
phi = np.linspace(0, np.pi, 30)

theta_grid, phi_grid = np.meshgrid(theta, phi)

r = float(Earth.equatorial_radius.in_meters())

x = r * np.cos(theta_grid) * np.sin(phi_grid)
y = r * np.sin(theta_grid) * np.sin(phi_grid)
z = r * np.cos(phi_grid)

earth = go.Surface(
    x = x,
    y = y,
    z = z,
    colorscale = 'Viridis',
    showscale = False
)

initial_orbit_trace = go.Scatter3d(
    x = initial_orbit_df['$x_{x}^{ECI}$'],
    y = initial_orbit_df['$x_{y}^{ECI}$'],
    z = initial_orbit_df['$x_{z}^{ECI}$'],
    mode = 'lines',
    marker = dict(
        size = 0,
        color = initial_orbit_df['$x_{z}^{ECI}$'],
        colorscale = 'Viridis',
        showscale = False
    ),
    line = dict(
        color = initial_orbit_df['$x_{z}^{ECI}$'],
        width = 3
    )
)

final_orbit_trace = go.Scatter3d(
    x = final_orbit_df['$x_{x}^{ECI}$'],
    y = final_orbit_df['$x_{y}^{ECI}$'],
    z = final_orbit_df['$x_{z}^{ECI}$'],
    mode = 'lines',
    marker = dict(
        size = 0,
        color = final_orbit_df['$x_{z}^{ECI}$'],
        colorscale = 'Viridis',
        showscale = False
    ),
    line = dict(
        color = final_orbit_df['$x_{z}^{ECI}$'],
        width = 3
    )
)

transfer_orbit_trace = go.Scatter3d(
    x = transfer_orbit_df['$x_{x}^{ECI}$'],
    y = transfer_orbit_df['$x_{y}^{ECI}$'],
    z = transfer_orbit_df['$x_{z}^{ECI}$'],
    mode = 'lines',
    marker = dict(
        size = 0,
        color = transfer_orbit_df['$x_{z}^{ECI}$'],
        colorscale = 'Viridis',
        showscale = False
    ),
    line = dict(
        color = transfer_orbit_df['$x_{z}^{ECI}$'],
        width = 3
    )
)

figure = go.Figure(
    data = [earth, initial_orbit_trace, final_orbit_trace, transfer_orbit_trace],
    layout = go.Layout(
        title = None,
        width = 800,
        height = 1000,
        showlegend = False,
        scene = go.layout.Scene(
            xaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            yaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            zaxis = dict(
                gridcolor = 'rgb(255, 255, 255)',
                zerolinecolor = 'rgb(255, 255, 255)',
                showbackground = True,
                backgroundcolor = 'rgb(230, 230,230)'
            ),
            camera = dict(
                up = dict(
                    x = 0,
                    y = 0,
                    z = 1
                ),
                eye = dict(
                    x = -1.7428,
                    y = 1.0707,
                    z = 0.7100,
                )
            ),
        )
    )
)

figure.show()

---